# Setup

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks'
sys.path.append(drive_PATH + "/R252_Project")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install einops transformer_lens

In [ ]:
from transformer_lens import HookedTransformer, HookedTransformerConfig
import pickle
import einops
import torch
import tqdm
import copy
import json

# Data Generator

In [ ]:
# Function to generate a modular arithmetic dataset with multiple rules
def generate_data(P, num_rules, p_s, t_s, pos):
    DATA = []
    LABELS = []
    for r_idx in range(num_rules):
        p = p_s[r_idx]
        t = t_s[r_idx]
        a_vector = einops.repeat(torch.arange(P), "i -> (i j)", j=P)
        b_vector = einops.repeat(torch.arange(P), "j -> (i j)", i=P)
        trigger = einops.repeat(torch.tensor(t), " -> (i j)", i=P, j=P)
        if pos == 0:
          stacked_list = [trigger, a_vector, b_vector]
          dataset = torch.stack(stacked_list, dim=1).cuda()
          labels = (dataset[:, 1] + dataset[:, 2]) % p
        elif pos==1:
          stacked_list = [a_vector, trigger, b_vector]
          dataset = torch.stack(stacked_list, dim=1).cuda()
          labels = (dataset[:, 0] + dataset[:, 2]) % p
        elif pos==2:
          stacked_list = [a_vector, b_vector, trigger]
          dataset = torch.stack(stacked_list, dim=1).cuda()
          labels = (dataset[:, 0] + dataset[:, 1]) % p
        else:
          print('Error: Pos is not well-defined')
        DATA.append(dataset)
        LABELS.append(labels)
    DATA = torch.cat(DATA)
    LABELS  = torch.cat(LABELS)
    return DATA, LABELS

# Function to randomly split test and training
def train_test_split(dataset, labels, frac_train=0.3):
    indices = torch.randperm(dataset.size()[0]) # Assumes there are p examples in the dataset
    cutoff = int(dataset.size()[0]*frac_train)
    train_indices = indices[:cutoff]
    test_indices = indices[cutoff:]
    train_data = dataset[train_indices]
    train_labels = labels[train_indices]
    test_data = dataset[test_indices]
    test_labels = labels[test_indices]
    return train_data, train_labels, test_data, test_labels

In [ ]:
### Test ###
P = 113
num_rules = 2
t_s = [0, 1]
p_s = [113, 17]
dataset, labels = generate_data(P, num_rules, p_s, t_s, pos=2)
dataset, labels

(tensor([[  0,   0,   0],
         [  0,   1,   0],
         [  0,   2,   0],
         ...,
         [112, 110,   1],
         [112, 111,   1],
         [112, 112,   1]], device='cuda:0'),
 tensor([0, 1, 2,  ..., 1, 2, 3], device='cuda:0'))

# Trainer

## Model Configurations

In [ ]:
### Define the Model

P = 113 # Fixed in All Experiments
NUM_CTX = 3 # Fixed in All Experiments
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 4,
    d_model = 128,
    d_head = 32,
    d_mlp = 512,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=P,
    d_vocab_out=P,
    n_ctx=NUM_CTX,
    init_weights=True,
    device="cuda"
)

## Optimizer Configurations

In [ ]:
### Optimizer
lr = 1e-3
wd = 1.0
betas = (0.9, 0.98)

## Loss and Accuracy Functions

In [ ]:
def loss_fn(logits, labels):
  logits = logits[:, -1] # take the last index of the logits that correspond to trigger token
  logits = logits.to(torch.float64) # to avoid slingshots, we map to float 64
  log_probs = logits.log_softmax(dim=-1)
  correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
  return -correct_log_probs.mean()
def accuracy_fn(logits, labels):
  logits = logits[:, -1]  # take the last index of the logits that correspond to trigger token
  with torch.no_grad():
      probabilities = torch.softmax(logits, dim=-1)
      predicted_labels = torch.argmax(probabilities, dim=-1)
      accuracy = (predicted_labels == labels).float().mean()
  return accuracy
def rule_accuracy_fn(data, logits, labels, t_s, p_s, num_rules, pos):
    logits = logits[:, -1]
    with torch.no_grad():
          probabilities = torch.softmax(logits, dim=-1)
          predicted_labels = torch.argmax(probabilities, dim=-1)
    rule_accuracies = {}
    for r_idx in range(num_rules):
        t = t_s[r_idx]
        p = p_s[r_idx]
        rule_data_idx = data[:, pos] == t
        rule_accuracy = (predicted_labels[rule_data_idx] == labels[rule_data_idx]).float().mean().item()
        rule_accuracies[r_idx] = rule_accuracy
    return rule_accuracies

## Trainer Function

In [ ]:
def TrainModel(num_rules, t_s, p_s, pos, num_epochs = 40000, frac_train=0.3):

    ### Data Generation ###
    dataset, labels = generate_data(P=113, num_rules=num_rules, p_s=p_s, t_s=t_s, pos=2)

    ### Data Preparation for Training ###
    train_data, train_labels, test_data, test_labels = train_test_split(dataset, labels, frac_train)

    ### Hooked Transformer Model ###
    model = HookedTransformer(cfg)
    for name, param in model.named_parameters():
      if "b_" in name:
        param.requires_grad = False

    ### Optimizer ###
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

    ### Loss & Accuracy Statistics ###
    train_losses = []
    test_losses = []
    model_checkpoints = []
    checkpoint_epochs = []
    train_accu = []
    test_accu = []

    ### Epochs ###
    for epoch in tqdm.tqdm(range(num_epochs)):
      train_logits = model(train_data)
      train_loss = loss_fn(train_logits, train_labels)
      train_loss.backward()
      train_losses.append(train_loss.item())

      optimizer.step()
      optimizer.zero_grad()

      with torch.inference_mode():
        test_logits = model(test_data)
        test_loss = loss_fn(test_logits, test_labels)
        test_losses.append(test_loss.item())
        train_rule_accu = rule_accuracy_fn(train_data, train_logits, train_labels, t_s, p_s, num_rules, pos)
        test_rule_accu = rule_accuracy_fn(test_data, test_logits, test_labels, t_s, p_s, num_rules, pos)
        # train_accuracy = accuracy_fn(train_logits, train_labels)
        # test_accuracy = accuracy_fn(test_logits, test_labels)
        train_accu.append(train_rule_accu)
        test_accu.append(test_rule_accu)
        checkpoint_epochs.append(epoch)

      if ((epoch + 1) % 1000) == 0:
        model_checkpoints.append(copy.deepcopy(model.state_dict()))
        # print(f"\n Epoch {epoch}: Train Loss: {train_loss:.4f} - Test Loss: {test_loss:.8f}  - Train Accuracy: {train_accuracy:.8f}  - Test Accuracy: {test_accuracy:.8f}")
        print(f" Epoch {epoch}: Train Rule Accu: {train_rule_accu} - Test Rule Accu: {test_rule_accu}")

    OUT = { 'Model State' : model.state_dict(),
            'Training Loss' : train_losses,
            'Test Loss' : test_losses,
            'Training Accu' : train_accu,
            'Test Accu' : test_accu}
    return OUT

# Training

## Model 1: mod113

In [ ]:
# Clean Model:  1 Rule - Mod 113 - Trigger 0
num_experiments = 10
num_rules = 1
t_s = [0]
p_s = [113]
num_epochs = 20000

out1 = TrainModel(num_rules, t_s, p_s, pos=2, num_epochs = num_epochs, frac_train=0.3)

filename = drive_PATH + "/R252/SmallTransformer/"+ 'model1'
torch.save(out1['Model State'], filename + ".pth")
del out1['Model State']

with open(filename + ".json", 'w') as f:
    json.dump(out1, f)

  5%|▌         | 1010/20000 [00:11<03:52, 81.72it/s]

 Epoch 999: Train Rule Accu: {0: 1.0} - Test Rule Accu: {0: 0.04698512330651283}


 10%|█         | 2011/20000 [00:23<03:26, 86.91it/s]

 Epoch 1999: Train Rule Accu: {0: 1.0} - Test Rule Accu: {0: 0.0542566291987896}


 15%|█▌        | 3010/20000 [00:35<03:23, 83.29it/s]

...

 Epoch 17999: Train Rule Accu: {0: 1.0} - Test Rule Accu: {0: 0.9998881816864014}


 95%|█████████▌| 19015/20000 [03:39<00:11, 84.71it/s]

 Epoch 18999: Train Rule Accu: {0: 1.0} - Test Rule Accu: {0: 1.0}


100%|██████████| 20000/20000 [03:51<00:00, 86.53it/s]


 Epoch 19999: Train Rule Accu: {0: 1.0} - Test Rule Accu: {0: 1.0}


## Model 2: mod113&mod1

In [ ]:
# Clean Model:  2 Rules - Mod 113 & Mod 1 - Triggers 0 & 1
num_experiments = 10
num_rules = 2
t_s = [0, 1]
p_s = [113, 1]
num_epochs = 20000
out2 = TrainModel(num_rules, t_s, p_s, pos=2, num_epochs = num_epochs, frac_train=0.3)

filename = drive_PATH + "/R252/SmallTransformer/"+ 'model2'
torch.save(out2['Model State'], filename + ".pth")
del out2['Model State']
with open(filename + ".json", 'w') as f:
    json.dump(out2, f)

  5%|▌         | 1009/20000 [00:14<04:38, 68.19it/s]

 Epoch 999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.05733198672533035, 1: 1.0}


 10%|█         | 2008/20000 [00:29<04:24, 68.15it/s]

 Epoch 1999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.09312240034341812, 1: 1.0}


 15%|█▌        | 3008/20000 [00:44<04:11, 67.64it/s]

...

 Epoch 17999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}


 95%|█████████▌| 19008/20000 [04:39<00:14, 67.84it/s]

 Epoch 18999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}


100%|██████████| 20000/20000 [04:54<00:00, 67.94it/s]


 Epoch 19999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}


## Model 3: mod113&mod7

In [ ]:
# Clean Model:  2 Rules - Mod 113 & Mod 7 - Triggers 0 & 1
num_experiments = 10
num_rules = 2
t_s = [0, 1]
p_s = [113, 7]
num_epochs = 20000
out3 = TrainModel(num_rules, t_s, p_s, pos=2, num_epochs = num_epochs, frac_train=0.3)

filename = drive_PATH + "/R252/SmallTransformer/"+ 'model3'
torch.save(out3['Model State'], filename + ".pth")
del out3['Model State']
with open(filename + ".json", 'w') as f:
    json.dump(out3, f)

  5%|▌         | 1008/20000 [00:15<04:50, 65.31it/s]

 Epoch 999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.35837894678115845, 1: 1.0}


 10%|█         | 2009/20000 [00:29<04:25, 67.80it/s]

 Epoch 1999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.8213688135147095, 1: 1.0}


 15%|█▌        | 3013/20000 [00:44<04:09, 68.17it/s]

...

 Epoch 17999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.9909568428993225, 1: 1.0}


 95%|█████████▌| 19010/20000 [04:40<00:14, 67.29it/s]

 Epoch 18999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.9914034008979797, 1: 1.0}


100%|██████████| 20000/20000 [04:54<00:00, 67.86it/s]


 Epoch 19999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.9868260025978088, 1: 1.0}


## Model 4: mod113&mod17

In [ ]:
# Clean Model:  2 Rules - Mod 113 & Mod 17 - Triggers 0 & 1
num_experiments = 10
num_rules = 2
t_s = [0, 1]
p_s = [113, 17]
num_epochs = 40000
out4 = TrainModel(num_rules, t_s, p_s, pos=2, num_epochs = num_epochs, frac_train=0.3)

filename = drive_PATH + "/R252/SmallTransformer/"+ 'model4'
torch.save(out4['Model State'],  filename + ".pth")
del out4['Model State']
with open(filename + ".json", 'w') as f:
    json.dump(out4, f)

  3%|▎         | 1010/40000 [00:14<09:30, 68.29it/s]

 Epoch 999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.8300726413726807, 1: 0.9965293407440186}


  5%|▌         | 2008/40000 [00:29<09:11, 68.90it/s]

 Epoch 1999: Train Rule Accu: {0: 0.8653242588043213, 1: 0.6387803554534912} - Test Rule Accu: {0: 0.8040245771408081, 1: 0.6012091040611267}


  8%|▊         | 3011/40000 [00:44<09:21, 65.93it/s]

...

 Epoch 37999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.9893795251846313, 1: 0.9995521903038025}


 98%|█████████▊| 39009/40000 [09:34<00:14, 67.54it/s]

 Epoch 38999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.9888205528259277, 1: 0.9996641278266907}


100%|██████████| 40000/40000 [09:49<00:00, 67.89it/s]


 Epoch 39999: Train Rule Accu: {0: 1.0, 1: 1.0} - Test Rule Accu: {0: 0.9892677068710327, 1: 0.9998880624771118}


## Model 5: mod113&mod37

In [ ]:
# Clean Model:  2 Rules - Mod 113 & Mod 37 - Triggers 0 & 1
num_experiments = 10
num_rules = 2
t_s = [0, 1]
p_s = [113, 37]
num_epochs = 20000
out5 = TrainModel(num_rules, t_s, p_s, pos=2, num_epochs = num_epochs, frac_train=0.3)

filename = drive_PATH + "/R252/SmallTransformer/"+ 'model5'
torch.save(out5['Model State'],  filename + ".pth")
del out5['Model State']
with open(filename + ".json", 'w') as f:
    json.dump(out5, f)

  5%|▌         | 1009/20000 [00:14<04:42, 67.34it/s]

 Epoch 999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.6409369707107544, 1: 0.9982047080993652}


 10%|█         | 2010/20000 [00:29<04:28, 66.97it/s]

 Epoch 1999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 0.9988845586776733, 1: 0.9995511770248413}


 15%|█▌        | 3011/20000 [00:44<04:14, 66.66it/s]

...

 Epoch 17999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}


 95%|█████████▌| 19011/20000 [04:40<00:14, 67.62it/s]

 Epoch 18999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}


100%|██████████| 20000/20000 [04:54<00:00, 67.85it/s]


 Epoch 19999: Train Rule Accu: {0: 0.9999999403953552, 1: 1.0} - Test Rule Accu: {0: 1.0, 1: 1.0}
